# Laboratorio 4: Clasificación II (comparar clasifidores, selección de hipérparámetros, clases desbalanceadas)

# Declaración de compromiso ético


Nosotros **AGREGUEN SUS NOMBRES COMPLETOS**, declaramos que realizamos de manera grupal los pasos de la presente actividad. También declaramos no incurrir en copia, ni compartir nuestras respuestas con otras personas ni con otros grupos. Por lo que, ratificamos que las respuestas son de nuestra propia confección y reflejan nuestro propio conocimiento.

# Instrucciones


1. Trabajen en equipos de dos personas. Salvo excepciones, no se corregirá entregas con menos de dos integrantes.

2. Modifique este archivo `.ipynb` agregando sus respuestas donde corresponda.

3. El formato de entrega para esta actividad es un archivo **html**. Genere un archivo HTML y súbalo a U-Cursos. Basta con que **uno de los integrantes haga la entrega**. Si ambos hacen una entrega en U-Cursos, se revisará cualquiera de éstas.

# Estructura del laboratorio

Este laboratorio está conformado por preguntas teóricas de temas vistos en clases y preguntas prácticas (donde se requiere completar código) intercaladas con preguntas de interpretación de resultados y análisis. La parte práctica se divide en:

1. Comparar clasificadores con ciertos *baselines* o clasificadores base.
2. Seleccionar hiperparámetros.
3. Trabajar con clases desbalanceadas.

## Pregunta 1.1  

Para realizar la evaluación de distintos clasificadores, vamos a crear la función `run_classifier()`, la cual evalúa un clasificador `clf` recibido como parámetro, un dataset `X,y` (features y target) y un número de tests llamado `num_test`. Esta función almacena y retorna los valores de precision, recall y f1-score en la variable `metrics` además de los resultados de predicción.

En base a lo anterior, incluya las sentencias que ajusten el modelo junto a su correspondiente predicción sobre los datos. **No use cross-validation ni tampoco el parámetro `random_state`.**


In [4]:
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
import numpy as np

def run_classifier(clf, X, y, num_tests=100):
    metrics = {'precision': [], 'recall': [], 'f1-score': []}

    for _ in range(num_tests):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30)
        ### INICIO COMPLETAR ACÁ

        #### TIP: entrene el modelo con los sets de entrenamientos, y
        #### cree la variable predictions con las predicciones del conjunto de testing

        # Entrenar modelo 

        predictions = clf.fit(X_train, y_train).predict(X_test)

        ### FIN COMPLETAR ACÁ
        # 0=Malignant, 1=Benign. In sklearn metrics, positive label is by default=1
        #metrics['y_pred'] = predictions
        metrics['precision'].append(precision_score(y_test, predictions, pos_label=0))
        metrics['recall'].append(recall_score(y_test, predictions, pos_label=0))
        metrics['f1-score'].append(f1_score(y_test, predictions, pos_label=0))
    return metrics

Luego de completar el código anterior, ejecute el siguiente bloque para comparar distintos clasificadores.
Usaremos un **dataset de cáncer de mamas** para evaluar. La información del dataset se puede encontrar en el siguiente link: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html

In [5]:
## ejecutar este código

from sklearn.datasets import load_breast_cancer
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB  # naive bayes
from sklearn.neighbors import KNeighborsClassifier #kNN
from sklearn.svm import SVC  # support vector machine

bc = load_breast_cancer()    # dataset cancer de mamas
X_bc = bc.data
y_bc = bc.target

c0 = ("Base Dummy", DummyClassifier(strategy='stratified'))
c1 = ("Decision Tree", DecisionTreeClassifier(max_depth=5))
c2 = ("Gaussian Naive Bayes", GaussianNB())
c3 = ("KNN", KNeighborsClassifier(n_neighbors=10))
c4 = ("Support Vector Machines", SVC())

classifiers = [c0, c1, c2, c3, c4]

results = {}
for name, clf in classifiers:
    metrics = run_classifier(clf, X_bc, y_bc)   # hay que implementarla en el bloque anterior.
    results[name] = metrics
    print("----------------")
    print("Resultados para clasificador: ", name)
    print("Precision promedio:", np.array(metrics['precision']).mean())
    print("Recall promedio:", np.array(metrics['recall']).mean())
    print("F1-score promedio:", np.array(metrics['f1-score']).mean())
    print("----------------\n\n")

----------------
Resultados para clasificador:  Base Dummy
Precision promedio: 0.37186465572485905
Recall promedio: 0.3716900186311729
F1-score promedio: 0.3697714788796518
----------------


----------------
Resultados para clasificador:  Decision Tree
Precision promedio: 0.9086960189292944
Recall promedio: 0.9016304302340054
F1-score promedio: 0.9043150176723701
----------------


----------------
Resultados para clasificador:  Gaussian Naive Bayes
Precision promedio: 0.9381667007796242
Recall promedio: 0.8931039044728963
F1-score promedio: 0.9143283025568928
----------------


----------------
Resultados para clasificador:  KNN
Precision promedio: 0.9398739695930357
Recall promedio: 0.8822293499262599
F1-score promedio: 0.9095045153978848
----------------


----------------
Resultados para clasificador:  Support Vector Machines
Precision promedio: 0.9682960035171932
Recall promedio: 0.7820968181280223
F1-score promedio: 0.8639960245637066
----------------




## Pregunta 1.2

Analizando los resultados obtenidos de cada clasificador, y basándose en las métricas calculadas. ¿Cuál es el mejor clasificador? ¿Qué métricas observó para tomar esa decisión y por qué? **considerando el problema que aborda**. Fundamente su respuesta.

(Considere *malignant* como clase positiva, y *benign* como clase negativa.)


**Respuesta**:

El mejor clasificador en base a los resultados tanto de recall como precisión es Naive Bayes, siendo aquel con F1-score más alto al querer comparar con otros métodos y realizar un trade off entre recall/precisión, en donde resulta ser un mejor método que KNN ya que gana más precisión de lo que se pierde en recall. 

Las métricas observadas fueron F1-score para el trade off entre métodos con precisión y recall similares entre sí pero con leves diferencuas (Naive Bayes, KNN, Decision Tree). Por otra parte, se analizo que si bien SVM tiene una precisión muy alta, su recall tiene un 20% de casos en los que no le indica a alguien con cancer de mama que su cancer es maligno, siendo descartado por ello, analogamente, Base Dummy fue descartado ya que posee una baja precision y recall, dando lugar a una gran cantidad de casos de falsos positivos y falsos negativos de cancer de mama.
 


## Pregunta 1.3

¿Cuál sería la razón de maximizar la métrica 'recall' en el problema abordado?

**Respuesta:**

# Parte 2: Seleccionar hiperparámetros

Los hiperparámetros son parámetros que no se aprenden directamente dentro de los estimadores. En scikit-learn se pasan como argumentos al constructor de las clases, por ejemplo, cuál kernel usar para Support Vector Classifier, o qué criterio para Decision Tree, etc. Es posible y recomendable buscar en el espacio de hiperparámetros la mejor alternativa.

Tenga en cuenta que es común que un pequeño subconjunto de esos parámetros pueda tener un gran impacto en el rendimiento predictivo o de cálculo del modelo, mientras que otros pueden dejar sus valores predeterminados. Se recomienda leer la documentación de la clase de estimador para obtener una mejor comprensión de su comportamiento esperado.


**Dataset:** En esta y la siguiente parte del laboratorio utilizaremos el dataset **"ML Classification: Predicting 5-Year Career Longevity for NBA Rookies"** de data.world (https://data.world/ssaudz/ml-classification-predicting-5-year-career-longevity-for-nb). Este dataset contiene estadísticas de los novatos en la NBA y busca predecir si un jugador podrá durar 5 años en la liga. La columna objetivo es *TARGET_5Yrs*. Esta es una versión preprocesada del dataset original (después de eliminar registros duplicados por nombre del jugador, anonimizar los datos, y eliminar los registros con valores nulos).

In [6]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/cinthiasanchez/data-mining/main/NBA_career_longevity.csv')
data.shape

(1284, 20)

In [7]:
data.head(2)

,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
0,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25.0,1.6,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,0
1,35,26.9,7.2,2.0,6.7,29.6,0.7,2.8,23.5,2.6,3.4,76.5,0.5,2.0,2.4,3.7,1.1,0.5,1.6,0


In [8]:
#separando atributos predictivos (X) del atributo objetivo (y)
X = data.iloc[:,:-1].values
y = data['TARGET_5Yrs'].values

#dividiendo los datos de entrenamiento y validación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30,
                                                    random_state=15, stratify=y)

## GridSearchCV

Una alternativa para seleccionar hiperparámetros es GridSearchCV, la cual considera exhaustivamente todas las combinaciones de parámetros. GridSearchCV recibe un `estimador`, `param_grid` (un diccionario o una lista de diccionarios con los nombres de los parámetros a probar como keys y una lista de los valores a probar), `scoring` una o varias funciones de puntuación (score) para evaluar cada combinación de parámetros (opciones válidas: https://scikit-learn.org/stable/modules/model_evaluation.html), y `cv` una extrategia para hacer validación cruzada.

El siguiente código muestra cómo seleccionar el número de vecinos y qué pesos otorgar a los vecinos en un clasificador KNN.

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Definimos una semilla para que los resultados sean reproducibles
np.random.seed(42)

#Configure tuned_parameters
tuned_parameters = {'n_neighbors': [1, 3, 5, 10],
                    'weights': ['uniform','distance']}

#set scoring metric
score = 'precision'

#Construir el clf con GridSearch
clf = GridSearchCV(
    KNeighborsClassifier(),
    param_grid=tuned_parameters,
    cv=5,
    scoring=score
)

#Entrenar clf
clf.fit(X_train, y_train)

print("Mejor combinación de parámetros:")
print(clf.best_params_)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Mejor combinación de parámetros:
{'n_neighbors': 10, 'weights': 'uniform'}
              precision    recall  f1-score   support

           0       0.64      0.60      0.62       145
           1       0.77      0.80      0.78       241

    accuracy                           0.73       386
   macro avg       0.71      0.70      0.70       386
weighted avg       0.72      0.73      0.72       386



## Pregunta 2.1

*  a) Utilizando los datos del bloque anterior (NBA_career_longevity.csv), realice este mismo proceso para un clasificador `DecisionTree` y los parametros `criterion=['gini','entropy']`, `max_depth=[3,5,7,10]` y tomando como `scoring` metric `'f1'`. Use `cv=10`.
*  b) ¿Qué puede decir de los resultados, con cuáles parámetros los obtuvo (revise que su respuesta concuerde con los resultados que imprime)? ¿Cuál considera que es la principal ventaja de aplicar GridSearchCV? ¿Considera que es necesario seguir explorando los parámetros?

**Respuesta de b)**:

In [25]:
## RESPUESTA A PREGUNTA 2.1 a)

### INICIO COMPLETAR ACÁ

# Definimos una semilla para que los resultados sean reproducibles
np.random.seed(39)

#Configure tuned_parameters

tuned_parameters = {'criterion': ['gini', 'entropy'],
                     'max_depth': [3, 5, 7, 10]}

#set scoring metric
score = 'f1'

#Construir el clf con GridSearch
clf = GridSearchCV(
    DecisionTreeClassifier(),
    param_grid=tuned_parameters,
    cv=10,
    scoring=score
)

#Entrenar clf
clf.fit(X_train, y_train)

# Recuerde definir una semilla para que los resultados sean reproducibles
# Use su número favorito como semilla

### FIN COMPLETAR ACÁ

print("Mejor combinación de parámetros:")
print(clf.best_params_)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

Mejor combinación de parámetros:
{'criterion': 'gini', 'max_depth': 3}
              precision    recall  f1-score   support

           0       0.62      0.59      0.61       145
           1       0.76      0.78      0.77       241

    accuracy                           0.71       386
   macro avg       0.69      0.69      0.69       386
weighted avg       0.71      0.71      0.71       386



*Respuesta:*

Para los parametros definidos, la acurracy da un valor bueno, pero no deseable ya que deja una gran margen de error, es por esto que lo óptimo seria seguir explorando los parámetros o incluso volver al dataset para entender mejor como mejorar el modelo. 

La ventaja de GridSearchCV es que permite obtener los mejores parametros a utilizar mediante una busqueda exhaustiva, mejorando la eficiencia del analisis, 

# Parte 3: Trabajar con clases desbalanceadas

Al explorar el dataset anterior, se nota un desbalance importante (38%-62%). Para mejorar el rendimiento de un clasificador sobre clases desbalanceadas existen varias técnicas. En esta parte, veremos cómo tratar con este problema usando (sub/over) sampling de las clases.

(*Nota: Para ejecutar el siguiente bloque es necesaria la librería `pandas` que viene incluida en Anaconda.*)

Note el desbalance de las clases ejecutando el siguiente código:

In [26]:
print("Distribucion de clases original")
data['TARGET_5Yrs'].value_counts()

Distribucion de clases original


TARGET_5Yrs
1    802
0    482
Name: count, dtype: int64

Antes de hacer algo para tratar el desbalance entre las clases primero debemos dividir en train-test. Como ya hicimos la partición de train y test, vamos a explorarla a continuación.

In [27]:
#Dividmos igual que arriba.
#Para facilitar el balance manual, X contendrá el dataset completo, pero luego eliminaremos de este el atributo objetivo.

data_train, data_test, ytrain, ytest = train_test_split(data, data['TARGET_5Yrs'], test_size=.30,
                                                random_state=15, stratify=data['TARGET_5Yrs'])

print("Cantidad de instancias por clase en el train:")
print("Clase 1: " + str((ytrain==1).sum()))
print("Clase 0: " + str((ytrain==0).sum()))

Cantidad de instancias por clase en el train:
Clase 1: 561
Clase 0: 337


Aplicaremos **oversampling** y **subsampling** al train para que queden balanceados. Ejecute el siguiente código y note ahora que las clases están balanceadas.

In [28]:
print("Distribución de clases usando (over/sub) sampling: \n")
data_train = data_train.reset_index(drop=True)

# oversampling sobre la clase 0
idx = np.random.choice(data_train[data_train['TARGET_5Yrs'] == 0].index, size=224)
data_oversampled = pd.concat([data_train, data_train.iloc[idx]])
print("Data oversampled on class '0'")
print(data_oversampled['TARGET_5Yrs'].value_counts())
print()


# subsampling sobre la clase 1
idx = np.random.choice(data_train.loc[data_train.TARGET_5Yrs == 1].index, size=224, replace=False)
data_subsampled = data_train.drop(data_train.iloc[idx].index)
print("Data subsampled on class '1'")
print(data_subsampled['TARGET_5Yrs'].value_counts())

Distribución de clases usando (over/sub) sampling: 

Data oversampled on class '0'
TARGET_5Yrs
1    561
0    561
Name: count, dtype: int64

Data subsampled on class '1'
TARGET_5Yrs
1    337
0    337
Name: count, dtype: int64


**Nota:** *Librerías como `imblearn` son muy útiles para balancear los datos.*

## Pregunta 3. 1

¿Por qué aplicar subsampling/oversampling de las clases sobre el conjunto de entrenamiento en lugar de aplicarlo sobre el dataset completo? Argumente su respuesta.

**Respuesta:**

Al realizar subsampling/oversampling en el dataset completo ocurre que por ejemplo se crean instancias que no existen y que pueden estar mal (por ejemplo un gato que vuela) o se pueden eliminar instancias correctas (un gato que maulla), afectanto así a los datos de testing.

Además, una de las finalidades de realizar subsampling/oversampling es que se normalice la cantidad de datos o las probabilidades si se usa Naive Bayes para el dataset de training, con la finalidad de que permita mejorar el modelo en el proceso de clasificación.


In [29]:
## ejecutar este código para preparar los datos
from sklearn.metrics import classification_report

# Preparando los data frames para ser compatibles con sklearn

# datos test (mismo para todos los conjuntos de entrenamiento)
X_test = data_test[data_train.columns[:-1]] # todo hasta la penultima columna
y_test = data_test[data_train.columns[-1]]  # la última columna

# datos entrenamiento "originales"
X_orig = data_train[data_train.columns[:-1]]
y_orig = data_train[data_train.columns[-1]]

# datos entrenamiento "oversampleados"
X_over = data_oversampled[data_train.columns[:-1]]
y_over = data_oversampled[data_train.columns[-1]]

# datos entrenamiento "subsampleados"
X_subs = data_subsampled[data_train.columns[:-1]]
y_subs = data_subsampled[data_train.columns[-1]]


## Pregunta 3.2

Complete el código necesario para entrenar un clasificador DecisionTree en cada uno de los tres casos (**original**, con **oversampling** y con **subsampling**) y luego compare los resultados sobre el conjunto de test (este es el mismo para los tres casos) obtenido con train_test_split sobre los datos originales. Muestre Precision, Recall y F1-score.

Emplee como datos de entrada lo del bloque anterior.

In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

## Pasos:
##  - instanciar el clasificador con DecisionTreeClassifier()
##  - entrenar con fit()
##  - hacer las predicciones
##  - mostrar precision, recall y f1-score con classification report.

# Fijar con random state

print("ORIGINAL::::::::::")
clf_orig = DecisionTreeClassifier()
clf_orig.fit(X_orig, y_orig)
pred_orig = clf_orig.predict(X_test)
print(classification_report(y_test, pred_orig))

### INICIO COMPLETAR ACÁ

print("SUBSAMPLED::::::::::")
clf_subs = DecisionTreeClassifier()
clf_subs.fit(X_subs, y_subs)
pred_subs = clf_subs.predict(X_test)
print(classification_report(y_test, pred_subs))

print("OVERSAMPLED::::::::::")
clf_over = DecisionTreeClassifier()
clf_over.fit(X_over, y_over)
pred_over = clf_over.predict(X_test)
print(classification_report(y_test, pred_over))


### FIN COMPLETAR ACÁ

ORIGINAL::::::::::
              precision    recall  f1-score   support

           0       0.52      0.48      0.50       145
           1       0.70      0.73      0.72       241

    accuracy                           0.64       386
   macro avg       0.61      0.61      0.61       386
weighted avg       0.63      0.64      0.64       386

SUBSAMPLED::::::::::
              precision    recall  f1-score   support

           0       0.50      0.66      0.57       145
           1       0.75      0.61      0.67       241

    accuracy                           0.63       386
   macro avg       0.63      0.63      0.62       386
weighted avg       0.66      0.63      0.63       386

OVERSAMPLED::::::::::
              precision    recall  f1-score   support

           0       0.52      0.52      0.52       145
           1       0.71      0.71      0.71       241

    accuracy                           0.64       386
   macro avg       0.62      0.62      0.62       386
weighted avg

## Pregunta 3.3

- Observe los resultados obtenidos por clase con cada conjunto de entrenamiento, ¿se puede observar alguna diferencia importante?
- Indique una desventaja de usar oversampling y una desventaja de usar subsampling en clasificación.

**Respuesta**: 
Para cada métrica podemos realizar las siguientes observaciones:
F1 Score clase 0: Original < Oversampling < Subsampling
F1 Score clase 1: Subsampling < Oversampling < Original

accuracy: Subsampling < Original = Oversampling (no hay mucha diferencia entre conjuntos)
precision clase 0: Subsampling < Original = Oversampling
precision clase 1: Original < Oversampling < Subsampling

macro precision: Original < Oversampling < Subsampling
macro recall: Original < Oversampling < Subsampling
macro f1-score: Original < Oversampling < Subsampling

weight f1-score: Subsampling < Original = Oversampling

Si bien se obtuvieron diferencias, estas son muy leves, y pueden variar dependiendo de la ejecución, el cambio más notable se puede apreciar mediante f1-score por clase, en donde para clase=0 es mejor realizar Subsampling, mientras que para la clase=1 es mejor mantener la data original a fin de obtener el mejor modelo.

Por otra parte es interesante notar que para cada conjunto los valores de precision, recall y f1-score son equivalentes respectivamente, y que de igual forma el f1-score para weighted avg es equivalente a la accuracy en cada conjunto.

Una desventaja de usar oversampling es que al crear datos artificiales en base a los originales, se corre el riesgo de provocar overfitting y que el modelo pierda la capacidad de generalizar.

Una desventaja de usar subsampling es la pérdidad de instancias capaces de brindar variedad de muestras a la clase, lo que generaría que se pierda la capacidad de clasificar correctamente una instancia I perteneciente a la clase C ya que no se ha entrenado el modelo con instancias similares a la I para dicha clase C.

## Pregunta 3.4

Compare los resultados del caso **ORIGINAL** (donde el clasificador usa los parámetros por defecto DecisionTreeClassifier()) versus el resultado de la pregunta 2.1 donde usa los mejores parámetros con GridSearchCV. ¿Qué opina de los resultados?

**Respuesta**:

ORIGINAL::::::::::
              precision    recall  f1-score   support

           0       0.52      0.48      0.50       145
           1       0.70      0.73      0.72       241

    accuracy                           0.64       386
   macro avg       0.61      0.61      0.61       386
weighted avg       0.63      0.64      0.64       386


Mejor combinación de parámetros:
{'criterion': 'gini', 'max_depth': 3}
              precision    recall  f1-score   support

           0       0.62      0.59      0.61       145
           1       0.76      0.78      0.77       241

    accuracy                           0.71       386
   macro avg       0.69      0.69      0.69       386
weighted avg       0.71      0.71      0.71       386


Se observa que para las clases las medidas de precision, recall y f1-score son notablemente mejores para GridSearchCV, lo cual se ve reflejado en que el resultado de la Pregunta 2.1 tiene mayor accuracy que el caso 'Original'. Abordando macro avg notamos que nuevamente se cumple que los valores obtenidos son mejores en el caso de GridSearchCV e igualmente para weighted avg. Por lo tanto la conclusión es que es mucho mejor optar por las indicaciones obtenidas al usar este método.

### Eso es todo! Que tengan buena semana!! :))

![footer](https://miro.medium.com/v2/resize:fit:828/format:webp/1*8jAT7ocXcZTw5TcyiUiNIA.jpeg)